## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-03-52-15 +0000,nyt,Nor’easter Brings Heavy Rain and Flooding as I...,https://www.nytimes.com/2025/10/12/us/noreaste...
1,2025-10-13-03-51-29 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/12/world/...
2,2025-10-13-03-50-24 +0000,nyt,A Tennessee Sheriff Becomes the Face of Grief ...,https://www.nytimes.com/2025/10/12/us/tennesse...
3,2025-10-13-03-49-25 +0000,nypost,21 nuns escape disastrous fire engulfing nearl...,https://nypost.com/2025/10/12/world-news/21-nu...
4,2025-10-13-03-49-10 +0000,nyt,Why Now? The Lost Chances to Reach a Hostage D...,https://www.nytimes.com/2025/10/12/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,26
41,china,13
376,hamas,10
32,deal,10
139,israel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,77
140,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,77
28,2025-10-13-00-41-23 +0000,bbc,Trump says 'war is over' in Gaza as he flies t...,https://www.bbc.com/news/articles/cn409y125v3o...,66
82,2025-10-12-20-10-31 +0000,nypost,Jake Sullivan begrudgingly credits Trump for I...,https://nypost.com/2025/10/12/us-news/jake-sul...,62
13,2025-10-13-02-15-18 +0000,nyt,Asia Markets Fall After Trump Threatens New Ta...,https://www.nytimes.com/2025/10/12/business/tr...,59


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,77,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...
184,45,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
53,40,2025-10-12-22-22-15 +0000,nypost,Israeli hostage release feels tantalizingly cl...,https://nypost.com/2025/10/12/us-news/hundreds...
97,30,2025-10-12-18-39-55 +0000,nypost,Kamala Harris reveals Biden sent her call to v...,https://nypost.com/2025/10/12/us-news/kamala-h...
143,29,2025-10-12-12-00-11 +0000,wapo,He was a homeless college student. He found ho...,https://www.washingtonpost.com/nation/2025/10/...
65,28,2025-10-12-21-42-29 +0000,nypost,Zelensky suggests Tomahawk missiles could make...,https://nypost.com/2025/10/12/us-news/zelensky...
50,27,2025-10-12-22-54-00 +0000,wsj,"Frozen in Time Since Hamas Attacked, a Kibbutz...",https://www.wsj.com/world/middle-east/frozen-i...
24,26,2025-10-13-01-03-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
7,24,2025-10-13-03-13-10 +0000,nypost,Wild video shows Texas airport worker lose con...,https://nypost.com/2025/10/12/us-news/wild-vid...
54,24,2025-10-12-22-17-55 +0000,bbc,Starmer arrives in Egypt ahead of summit on Ga...,https://www.bbc.com/news/articles/c9qnqx7znqno...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
